# Variational Quantum Eigensolver

In [ ]:
!pip install qiskit
!pip install qiskit-nature[pyscf] -U

In [ ]:
!pip install qiskit_aer

## Example: $ H_2 $

In [4]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit.circuit.library import TwoLocal
from typing import Tuple

In [18]:
def get_mapper_and_ansatz(ansatz_type: str):
    """
    Returns JordanWignerMapper and the UCCSD ansatz configured
    for the OH molecule

    Args:
        ansatz_type: 'UCCSD' or 'TwoLocal'

    Returns:
        es_problem: ElectronicStructureProblem
        mapper: JordanWignerMapper
        ansatz: UCCSD type ansatz for OH 
    """

    driver = PySCFDriver(
        atom="""H 0 0 0; H 0.0 0.0 0.9""",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )

    es_problem = driver.run()
    mapper = JordanWignerMapper()

    if ansatz_type == 'UCCSD':
        ansatz = UCCSD(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
            initial_state=HartreeFock(
                es_problem.num_spatial_orbitals,
                es_problem.num_particles,
                mapper,
            ),
        )
    elif ansatz_type == 'TwoLocal':
        ansatz = TwoLocal(
            rotation_blocks=["h", "rx"],
            entanglement_blocks="cz",
            entanglement="full",
            reps=2,
            parameter_prefix="y",
        )
    else:
        raise ValueError('Ansatz type not supported') 
    
    return es_problem, mapper, ansatz

In [19]:
from qiskit.utils import QuantumInstance, algorithm_globals

ansatz_type = 'UCCSD'
device = 'CPU'
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
iterations = 125
shot = 6000

/tmp/ipykernel_5129/3642115839.py:6: DeprecationWarning: The property ``qiskit.utils.algorithm_globals.QiskitAlgorithmGlobals.random_seed`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. This algorithm utility has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. You can run ``pip install qiskit_algorithms`` and import ``from qiskit_algorithms.utils`` instead. 
  algorithm_globals.random_seed = seeds


In [20]:
es_problem, mapper, ansatz = get_mapper_and_ansatz(ansatz_type)

/home/sorana/.conda/envs/iccad-learn/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [21]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator

estimator = Estimator(
    backend_options = {
        'method': 'statevector',
        'device': device
        # 'noise_model': noise_model
    },
    run_options = {
        'shots': shot,
        'seed': seeds,
    },
    transpile_options = {
        'seed_transpiler':seed_transpiler
    }
)

/tmp/ipykernel_5129/3831695330.py:1: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.minimum_eigensolvers import VQE
/tmp/ipykernel_5129/3831695330.py:5: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstmator from qiskit.primitives.
  estimator = Estimator(


In [33]:
ansatz.decompose(reps=3).draw()

┌──────────┐┌─────┐┌───┐┌───┐┌──────────────┐┌───┐┌───┐ ┌───┐ ┌───┐┌───┐»
q_0: ┤ U(π,0,π) ├┤ Sdg ├┤ H ├┤ X ├┤ Rz(1.0*t[0]) ├┤ X ├┤ H ├─┤ S ├─┤ H ├┤ X ├»
     └──┬───┬───┘└─────┘└───┘└─┬─┘└──────────────┘└─┬─┘├───┤┌┴───┴┐├───┤└─┬─┘»
q_1: ───┤ H ├──────────────────■────────────────────■──┤ H ├┤ Sdg ├┤ H ├──■──»
     ┌──┴───┴───┐┌─────┐┌───┐┌───┐┌──────────────┐┌───┐├───┤└┬───┬┘├───┤┌───┐»
q_2: ┤ U(π,0,π) ├┤ Sdg ├┤ H ├┤ X ├┤ Rz(1.0*t[1]) ├┤ X ├┤ H ├─┤ S ├─┤ H ├┤ X ├»
     └──┬───┬───┘└─────┘└───┘└─┬─┘└──────────────┘└─┬─┘├───┤┌┴───┴┐├───┤└─┬─┘»
q_3: ───┤ H ├──────────────────■────────────────────■──┤ H ├┤ Sdg ├┤ H ├──■──»
        └───┘                                          └───┘└─────┘└───┘     »
«     ┌───────────────┐┌───┐┌───┐┌─────┐ ┌───┐                ┌───┐»
«q_0: ┤ Rz(-1.0*t[0]) ├┤ X ├┤ H ├┤ Sdg ├─┤ H ├────────────────┤ X ├»
«     └───────────────┘└─┬─┘├───┤└┬───┬┘ ├───┤           ┌───┐└─┬─┘»
«q_1: ───────────────────■──┤ H ├─┤ S ├──┤ H ├───────────┤ X ├──■──»
«     ┌───────────────┐┌───┐├───┤┌┴───┴┐ ├───┤      ┌───┐└─┬─┘     »
«q_2: ┤ Rz(-1.0*t[1]) ├┤ X ├┤ H ├┤ Sdg ├─┤ H ├──────┤ X ├──■───────»
«     └───────────────┘└─┬─┘├───┤└┬───┬┘┌┴───┴┐┌───┐└─┬─┘          »
«q_3: ───────────────────■──┤ H ├─┤ S ├─┤ Sdg ├┤ H ├──■────────────»
«                           └───┘ └───┘ └─────┘└───┘               »
«     ┌────────────────┐┌───┐┌───┐┌───┐┌─────┐┌───┐                      ┌───┐»
«q_0: ┤ Rz(-0.25*t[2]) ├┤ X ├┤ H ├┤ S ├┤ Sdg ├┤ H ├──────────────────────┤ X ├»
«     └────────────────┘└─┬─┘├───┤├───┤├─────┤├───┤                 ┌───┐└─┬─┘»
«q_1: ────────────────────■──┤ X ├┤ H ├┤ Sdg ├┤ H ├─────────────────┤ X ├──■──»
«                            └─┬─┘├───┤└┬───┬┘├───┤┌─────┐┌───┐┌───┐└─┬─┘     »
«q_2: ─────────────────────────■──┤ X ├─┤ H ├─┤ S ├┤ Sdg ├┤ H ├┤ X ├──■───────»
«                                 └─┬─┘ ├───┤ ├───┤└┬───┬┘└───┘└─┬─┘          »
«q_3: ──────────────────────────────■───┤ H ├─┤ S ├─┤ H ├────────■────────────»
«                                       └───┘ └───┘ └───┘                     »
«     ┌────────────────┐┌───┐┌───┐┌───┐┌─────┐┌───┐               ┌───┐»
«q_0: ┤ Rz(-0.25*t[2]) ├┤ X ├┤ H ├┤ S ├┤ Sdg ├┤ H ├───────────────┤ X ├»
«     └────────────────┘└─┬─┘├───┤├───┤└┬───┬┘├───┤          ┌───┐└─┬─┘»
«q_1: ────────────────────■──┤ X ├┤ H ├─┤ S ├─┤ H ├──────────┤ X ├──■──»
«                            └─┬─┘├───┤ ├───┤ ├───┤┌───┐┌───┐└─┬─┘     »
«q_2: ─────────────────────────■──┤ X ├─┤ H ├─┤ S ├┤ H ├┤ X ├──■───────»
«                                 └─┬─┘ ├───┤ ├───┤└───┘└─┬─┘          »
«q_3: ──────────────────────────────■───┤ H ├─┤ H ├───────■────────────»
«                                       └───┘ └───┘                    »
«     ┌────────────────┐┌───┐┌───┐┌───┐┌─────┐ ┌───┐                ┌───┐»
«q_0: ┤ Rz(-0.25*t[2]) ├┤ X ├┤ H ├┤ S ├┤ Sdg ├─┤ H ├────────────────┤ X ├»
«     └────────────────┘└─┬─┘├───┤├───┤├─────┤ ├───┤           ┌───┐└─┬─┘»
«q_1: ────────────────────■──┤ X ├┤ H ├┤ Sdg ├─┤ H ├───────────┤ X ├──■──»
«                            └─┬─┘├───┤└┬───┬┘ ├───┤      ┌───┐└─┬─┘     »
«q_2: ─────────────────────────■──┤ X ├─┤ H ├──┤ H ├──────┤ X ├──■───────»
«                                 └─┬─┘ ├───┤ ┌┴───┴┐┌───┐└─┬─┘          »
«q_3: ──────────────────────────────■───┤ H ├─┤ Sdg ├┤ H ├──■────────────»
«                                       └───┘ └─────┘└───┘               »
«     ┌───────────────┐┌───┐┌───┐┌───┐┌───┐                      ┌───┐»
«q_0: ┤ Rz(0.25*t[2]) ├┤ X ├┤ H ├┤ S ├┤ H ├──────────────────────┤ X ├»
«     └───────────────┘└─┬─┘├───┤├───┤├───┤ ┌───┐           ┌───┐└─┬─┘»
«q_1: ───────────────────■──┤ X ├┤ H ├┤ S ├─┤ H ├───────────┤ X ├──■──»
«                           └─┬─┘├───┤├───┤┌┴───┴┐┌───┐┌───┐└─┬─┘     »
«q_2: ────────────────────────■──┤ X ├┤ H ├┤ Sdg ├┤ H ├┤ X ├──■───────»
«                                └─┬─┘├───┤└┬───┬┘├───┤└─┬─┘          »
«q_3: ─────────────────────────────■──┤ H ├─┤ S ├─┤ H ├──■────────────»
«                                     └───┘ └

In [22]:

vqe = VQE(estimator, ansatz, SLSQP())
vqe.initial_point = [0.0] * ansatz.num_parameters


In [23]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(mapper, vqe)


In [24]:
res = calc.solve(es_problem)

In [25]:
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.678745289658
  - computed part:      -1.678745289658
~ Nuclear repulsion energy (Hartree): 0.5879746788
> Total ground state energy (Hartree): -1.090770610858
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: -0.010 S^2: -0.009 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.70075351]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.702630979461]
    - computed part:      [0.0  0.0  1.702630979461]
  > Dipole moment (a.u.): [0.0  0.0  -0.001877469461]  Total: 0.001877469461
                 (debye): [0.0  0.0  -0.004772050925]  Total: 0.004772050925
 


In [34]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
calc = GroundStateEigensolver(mapper, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.7085349601
  - computed part:      -1.7085349601
~ Nuclear repulsion energy (Hartree): 0.5879746788
> Total ground state energy (Hartree): -1.1205602813
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.70075351]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.700753512109]
    - computed part:      [0.0  0.0  1.700753512109]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000002109]  Total: 0.000000002109
                 (debye): [0.0  0.0  -0.000000005359]  Total: 0.000000005359
 


### References:

\[1] - Use VQE to calculate the ground energy of hydrogen molecules on IBM Quantum https://arxiv.org/pdf/2305.06538.pdf